### 1. Import packages and dataset

In [1]:
import pandas as pd
from src.cleanfuncs import extract_states_dict, parse_snake_case

In [2]:
shootings = pd.read_csv("../input/fatal-police-shootings-data.csv", na_values={"armed": "undetermined", "threat_level": "undetermined"})

### 2. Exploratory Analysis

In [3]:
shootings.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True


In [4]:
shootings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5610 entries, 0 to 5609
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5610 non-null   int64  
 1   name                     5405 non-null   object 
 2   date                     5610 non-null   object 
 3   manner_of_death          5610 non-null   object 
 4   armed                    5226 non-null   object 
 5   age                      5360 non-null   float64
 6   gender                   5609 non-null   object 
 7   race                     4997 non-null   object 
 8   city                     5610 non-null   object 
 9   state                    5610 non-null   object 
 10  signs_of_mental_illness  5610 non-null   bool   
 11  threat_level             5362 non-null   object 
 12  flee                     5337 non-null   object 
 13  body_camera              5610 non-null   bool   
 14  longitude               

In [5]:
shootings.isnull().sum()

id                           0
name                       205
date                         0
manner_of_death              0
armed                      384
age                        250
gender                       1
race                       613
city                         0
state                        0
signs_of_mental_illness      0
threat_level               248
flee                       273
body_camera                  0
longitude                  269
latitude                   269
is_geocoding_exact           0
dtype: int64

In [6]:
shootings["manner_of_death"].value_counts()

shot                5331
shot and Tasered     279
Name: manner_of_death, dtype: int64

In [7]:
shootings["armed"].value_counts()

gun               3183
knife              828
unarmed            356
toy weapon         193
vehicle            161
                  ... 
metal rake           1
bean-bag gun         1
barstool             1
gun and sword        1
cordless drill       1
Name: armed, Length: 94, dtype: int64

In [8]:
shootings["armed"].value_counts().tail(20)

spear                  1
tire iron              1
air pistol             1
motorcycle             1
vehicle and machete    1
BB gun and vehicle     1
stapler                1
metal hand tool        1
bow and arrow          1
walking stick          1
car, knife and mace    1
oar                    1
ice pick               1
nail gun               1
contractor's level     1
metal rake             1
bean-bag gun           1
barstool               1
gun and sword          1
cordless drill         1
Name: armed, dtype: int64

In [9]:
shootings["race"].unique()

array(['A', 'W', 'H', 'B', 'O', nan, 'N'], dtype=object)

In [10]:
shootings["city"].value_counts().tail(10)

Ruth               1
Canaan Township    1
North St. Louis    1
Bremerton          1
Rosedale           1
Minot              1
Logan County       1
Willoughby         1
McIntosh County    1
Cape Canaveral     1
Name: city, dtype: int64

In [11]:
shootings["state"].unique()

array(['WA', 'OR', 'KS', 'CA', 'CO', 'OK', 'AZ', 'IA', 'PA', 'TX', 'OH',
       'LA', 'MT', 'UT', 'AR', 'IL', 'NV', 'NM', 'MN', 'MO', 'VA', 'NJ',
       'IN', 'KY', 'MA', 'NH', 'FL', 'ID', 'MD', 'NE', 'MI', 'GA', 'TN',
       'NC', 'AK', 'NY', 'ME', 'AL', 'MS', 'WI', 'SC', 'DE', 'DC', 'WV',
       'HI', 'WY', 'ND', 'CT', 'SD', 'VT', 'RI'], dtype=object)

In [12]:
shootings["signs_of_mental_illness"].value_counts()

False    4356
True     1254
Name: signs_of_mental_illness, dtype: int64

In [13]:
shootings["threat_level"].unique()

array(['attack', 'other', nan], dtype=object)

In [14]:
shootings["flee"].unique()

array(['Not fleeing', 'Car', 'Foot', 'Other', nan], dtype=object)

In [15]:
shootings["body_camera"].value_counts()

False    4951
True      659
Name: body_camera, dtype: int64

In [16]:
shootings["is_geocoding_exact"].unique()

array([ True, False])

In [17]:
# search for rows with highest number of null values
shootings.isnull().sum(axis=1).sort_values(ascending=False)

2860    7
5362    7
3152    6
3172    6
3840    6
       ..
3280    0
3279    0
3276    0
3275    0
0       0
Length: 5610, dtype: int64

### 3. Cleaning
#### Drop unuseful columns.

In [18]:
shootings.drop(columns=["id", "name"], inplace=True)

#### Fix columns with lowercase values

In [19]:
for column in ["manner_of_death", "armed", "threat_level"]:
    shootings[column] = shootings[column].str.title()
# fix uppercase "And"
shootings.replace({"manner_of_death": {r"And": "and"}}, regex=True, inplace=True)

#### Keep rows with at least 75% non-null values

In [20]:
shootings.dropna(thresh=int(shootings.shape[1]*0.75), inplace=True)

#### Replace gender, race and state columns with full names

In [21]:
# gender
gender_dict = {"M": "Male", "F": "Female"}
# race
race_keys = sorted(shootings.loc[shootings["race"].notnull(), "race"].unique())
race_values = sorted(["White, non-Hispanic", "Black, non-Hispanic", "Asian", "Native American", "Hispanic", "Other"])
race_dict = {k: v for k, v in zip(race_keys, race_values)}
# state
state_dict = extract_states_dict(shootings["state"])

shootings.replace({"gender": gender_dict, "race": race_dict, "state": state_dict}, inplace=True)

#### Parse column names from snake case

In [24]:
shootings.columns = [parse_snake_case(col) for col in shootings.columns]

#### Show final dataframe and export to `.csv`

In [25]:
shootings.head()

,Date,Manner of Death,Armed,Age,Gender,Race,City,State,Signs of Mental Illness,Threat Level,Flee,Body Camera,Longitude,Latitude,Is Geocoding Exact
0,2015-01-02,Shot,Gun,53.0,Male,Asian,Shelton,Washington,True,Attack,Not fleeing,False,-123.122,47.247,True
1,2015-01-02,Shot,Gun,47.0,Male,"White, non-Hispanic",Aloha,Oregon,False,Attack,Not fleeing,False,-122.892,45.487,True
2,2015-01-03,Shot and Tasered,Unarmed,23.0,Male,Hispanic,Wichita,Kansas,False,Other,Not fleeing,False,-97.281,37.695,True
3,2015-01-04,Shot,Toy Weapon,32.0,Male,"White, non-Hispanic",San Francisco,California,True,Attack,Not fleeing,False,-122.422,37.763,True
4,2015-01-04,Shot,Nail Gun,39.0,Male,Hispanic,Evans,Colorado,False,Attack,Not fleeing,False,-104.692,40.384,True


In [26]:
shootings.to_csv("../output/shootings_clean.csv")